In [21]:
!git clone https://github.com/OmarMoMorgan/DL_init.git

Cloning into 'DL_init'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 39 (delta 14), reused 31 (delta 6), pack-reused 0
Unpacking objects: 100% (39/39), 6.11 KiB | 782.00 KiB/s, done.


In [3]:
from torchvision.datasets import CIFAR10
from  torch.utils.data import DataLoader,Dataset
from torch import nn , optim
import torch
import pandas as pd


In [4]:
import sys
sys.path.append('/kaggle/working/DL_init')
from models import build_model
from tools import EarlyStopping, tune_model
from data import train_test_split_ , perPixel_mean_std, perChannel_mean_std, build_transforms

In [5]:
class MyDataset(Dataset):
    def __init__(self,x,y,transforms = None):
        self.x = x
        self.y = y
        self.transforms = transforms
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        if self.transforms is not None:
            x= self.transforms(image= self.x[idx])['image']
            return x.type(torch.float32),self.y[idx]

        return self.x[idx],self.y[idx]

In [6]:
train_data  = CIFAR10(root='./data',train = True,download = True)
test_data = CIFAR10(root='/data',train=False,download = True)


100%|██████████| 170498071/170498071 [00:01<00:00, 104883957.24it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 170498071/170498071 [00:03<00:00, 53961293.06it/s] 


Extracting /data/cifar-10-python.tar.gz to /data


In [7]:
# make a validation set
config= {'random_state' : 42,
          'shuffle':True,
         'train_size':0.9 }
X_train, X_val, y_train, y_val= train_test_split_(train_data.data,train_data.targets,config)
X_train.shape,X_val.shape

((45000, 32, 32, 3), (5000, 32, 32, 3))

```python
# stratify to keep the ratio 0.9 for every class in the trainined and testing
import numpy as np
uni, counts = np.unique(y_val, return_counts=True)
counts
```

In [8]:
mean_,std_ = perChannel_mean_std(X_train,'NHWC')
mean_,std_

(array([125.24002457, 122.93835269, 113.84449184]),
 array([62.95162973, 62.07468216, 66.70886545]))

In [9]:
train_tansforms,test_transforms = build_transforms('pipeline1',{'mean':mean_,
                                                                 'std':std_})

In [10]:
trainset = MyDataset(X_train,y_train,train_tansforms)
trainloader_ = DataLoader(trainset,batch_size=128, shuffle=True)

valset = MyDataset(X_val,y_val,test_transforms)
valloader_ = DataLoader(trainset,batch_size=1024, shuffle=False)

testset = MyDataset(test_data.data,test_data.targets,test_transforms)
testloader_ = DataLoader(testset,batch_size=512, shuffle=False)

In [11]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## For n = 3

In [12]:
model = build_model('sub_CNN',[3,3,3])
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [13]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.655 || acc 0.771 || Testing Loss 0.623 || Test acc 0.784
For epoch   9 || Training Loss 0.474 || acc 0.835 || Testing Loss 0.477 || Test acc 0.835


KeyboardInterrupt: 

In [ ]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './resnet3')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('resnet3.csv', index=False)

except:
    print('failed')



## For n = 5

In [22]:
model = build_model('sub_CNN',[3,3,3],3)
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

ValueError: out_channels must be divisible by groups

In [ ]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

In [ ]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './resnet5')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('resnet5.csv', index=False)

except:
    print('failed')



# fine tunning
in this part we shall try to fine tune a model based on a already pre trained model\
I will use for this part **Effiecent net** \
part 1: freeze all layers excpet the last one 

In [15]:
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
weights = EfficientNet_B3_Weights.DEFAULT
model_freeze = efficientnet_b3(weights=weights)
for param in model_freeze.parameters():
    param.requires_grad = False
model_freeze.classifier[1] = nn.Linear(1536,10)
print(model_freeze)
print(model_freeze.classifier[1].weight.requires_grad)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [ ]:

model_freeze.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_freeze.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [ ]:
hist_= tune_model(100,model_freeze,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

part 2: update all the layers 

In [ ]:
weights = EfficientNet_B3_Weights.DEFAULT
model_free = efficientnet_b3(weights=weights)
model_free.classifier[1] = nn.Linear(1536,10)
model_free.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_free.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [ ]:
hist_= tune_model(100,model_free,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)